In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import open3d as o3d
from plyfile import PlyData, PlyElement

In [2]:
# Read in image position data
with open('data/maps/image_poses.ply', 'rb') as f:
    plydata = PlyData.read(f)

print(plydata.elements)

(PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('scalar_image_identifier', 'int'), PlyProperty('scalar_time', 'double'), PlyProperty('scalar_roll', 'float'), PlyProperty('scalar_pitch', 'float'), PlyProperty('scalar_yaw', 'float'), PlyProperty('scalar_confidence', 'float')), count=1007, comments=[]),)


In [3]:
# Extract vertex data
vertices = plydata['vertex']
vertex_data = pd.DataFrame(vertices.data)

print(len(vertex_data))
vertex_data.head()

1007


,x,y,z,scalar_image_identifier,scalar_time,scalar_roll,scalar_pitch,scalar_yaw,scalar_confidence
0,0.008326,0.002601,0.002250,1,5.925832,0.110001,0.165191,0.000277,375240.65625
1,0.013624,0.011296,0.005206,2,7.425827,0.110000,0.165135,0.000398,419688.18750
2,0.019117,0.014602,0.006244,3,8.425823,0.109964,0.165150,0.000494,441757.87500
3,0.018069,0.015843,0.006658,4,9.425818,0.109810,0.165114,0.000559,472025.50000
4,0.023864,0.019839,0.006304,5,10.425815,0.110004,0.165117,0.000588,441257.71875


In [4]:
def get_image_name(row):
    return f'color_image_{int(row["scalar_image_identifier"])}.jpg'

vertex_data['image_name'] = vertex_data.apply(get_image_name, axis=1)

vertex_data

,x,y,z,scalar_image_identifier,scalar_time,scalar_roll,scalar_pitch,scalar_yaw,scalar_confidence,image_name
0,0.008326,0.002601,0.002250,1,5.925832,0.110001,0.165191,0.000277,375240.656250,color_image_1.jpg
1,0.013624,0.011296,0.005206,2,7.425827,0.110000,0.165135,0.000398,419688.187500,color_image_2.jpg
2,0.019117,0.014602,0.006244,3,8.425823,0.109964,0.165150,0.000494,441757.875000,color_image_3.jpg
3,0.018069,0.015843,0.006658,4,9.425818,0.109810,0.165114,0.000559,472025.500000,color_image_4.jpg
4,0.023864,0.019839,0.006304,5,10.425815,0.110004,0.165117,0.000588,441257.718750,color_image_5.jpg
...,...,...,...,...,...,...,...,...,...,...
1002,463.925659,-734.022705,32.844139,1003,1017.421082,0.033122,0.086241,-0.027662,164558.437500,color_image_1003.jpg
1003,463.920288,-734.019226,32.842110,1004,1018.421082,0.032466,0.086264,-0.027648,195473.328125,color_image_1004.jpg
1004,463.933228,-734.015625,32.852997,1005,1019.421082,0.031911,0.087616,-0.027664,200592.828125,color_image_1005.jpg
1005,463.940186,-734.014771,32.853302,1006,1020.421082,0.031673,0.088496,-0.027527,241922.437500,color_image_1006.jpg


In [ ]:
# Make a dataframe 

In [ ]:
# Detect cars in image with PyTorch

def detect_cars(image, model, device, min_score_threshold=0.5):
    """Detects cars in image with PyTorch model.

    Args:
        image (np.array): Image to detect cars in.
        model (torchvision.models.detection): PyTorch model.
        device (torch.device): Device to run model on.
        min_score_threshold (float): Minimum score threshold for car detection.

    Returns:
        np.array: Image with detected cars.
    """
    # Convert image to tensor
    image_tensor = torch.from_numpy(image).to(device)
    image_tensor = image_tensor.permute(2, 0, 1).float()
    image_tensor = image_tensor.unsqueeze(0)

    # Run model
    model.eval()
    with torch.no_grad():
        predictions = model(image_tensor)[0]

    # Filter predictions
    predictions = predictions[predictions['scores'] > min_score_threshold]

    # Draw bounding boxes
    image = image.copy()
    for box, score in zip(predictions['boxes'], predictions['scores']):
        box = box.cpu().numpy().astype(int)
        image = cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    return image